# Gerando as respostas SQL do modelo base utilizando few shot

In [ ]:
!pip install -U -q fsspec datasets transformers huggingface_hub bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import random
import numpy as np
import torch

seed = 21
random.seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

print("Random Python:", random.random())
print("Random NumPy:", np.random.rand())

Random Python: 0.16494947983319797
Random NumPy: 0.04872488080912729


Pegando a base do spider

In [ ]:
from datasets import load_dataset

spider_dataset = load_dataset("spider")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.51k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/831k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [ ]:
print(spider_dataset)

DatasetDict({
    train: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 1034
    })
})


In [ ]:
spider_dataset['train']

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 7000
})

In [ ]:
spider_dataset['train']['query'][0]

'SELECT count(*) FROM head WHERE age  >  56'

In [ ]:
spider_dataset['train']['question'][0]

'How many heads of the departments are older than 56 ?'

**IMPORTANTE:** Troque "hf_YOUR_TOKEN_HERE" pelo seu próprio token do hugging face. Para ter acesso ao mistral, você precisa concordar com os termos no site

https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

In [ ]:
from huggingface_hub import login

# Substitua 'hf_YOUR_TOKEN_HERE' pelo seu token real do Hugging Face
login(token='hf_YOUR_TOKEN_HERE')

Baixando o modelo (quantizamos ele com 8 bits para caber na GPU)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_8bit_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_8bit_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Exemplo de prompt

In [ ]:
text = f"""[INST]
You are an AI specialized in Text-to-SQL activity.
Based on a textual question, generate ONLY the respective SQL command.

Example 1
Question: {spider_dataset['train']['question'][0]}
Answer: {spider_dataset['train']['query'][0]}

Example 2
Question: {spider_dataset['train']['question'][3500]}
Answer: {spider_dataset['train']['query'][3500]}

Example 3
Question: {spider_dataset['train']['question'][-1]}
Answer: {spider_dataset['train']['query'][-1]}

Answer this question
Question: {spider_dataset['validation']['question'][0]}
[/INST]"""
print(text)

[INST]
You are an AI specialized in Text-to-SQL activity.
Based on a textual question, generate ONLY the respective SQL command.

Example 1
Question: How many heads of the departments are older than 56 ?
Answer: SELECT count(*) FROM head WHERE age  >  56

Example 2
Question: What are the full names and hire dates for employees in the same department as someone with the first name Clara?
Answer: SELECT first_name ,  last_name ,  hire_date FROM employees WHERE department_id  =  (SELECT department_id FROM employees WHERE first_name  =  "Clara")

Example 3
Question: What are all company names that have a corresponding movie directed in the year 1999?
Answer: SELECT T2.company_name FROM movie AS T1 JOIN culture_company AS T2 ON T1.movie_id  =  T2.movie_id WHERE T1.year  =  1999

Answer this question
Question: How many singers do we have?
[/INST]


In [ ]:
def generate_response(question):
  text = f"""[INST]
You are an AI specialized in Text-to-SQL activity.
Based on a textual question, generate ONLY the respective SQL command.

Example 1
Question: {spider_dataset['train']['question'][0]}
Answer: {spider_dataset['train']['query'][0]}

Example 2
Question: {spider_dataset['train']['question'][3500]}
Answer: {spider_dataset['train']['query'][3500]}

Example 3
Question: {spider_dataset['train']['question'][-1]}
Answer: {spider_dataset['train']['query'][-1]}

Answer this question
Question: {question}
[/INST]"""

  input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)

  generated_ids = model.generate(
    input_ids,
    max_new_tokens=1000,
    do_sample=True,
    temperature=0.01,
    pad_token_id=tokenizer.pad_token_id
  )

  full_response = tokenizer.decode(generated_ids[0], skip_special_tokens=False)

  #encontrando a resposta do modelo
  start_of_response_tag = "[/INST]"
  start_index = full_response.find(start_of_response_tag)

  try:
    # A resposta real começa logo após o [/INST] e um possível espaço.
    # Adicione o comprimento do tag e do espaço para pegar o início da resposta.
    model_raw_answer = full_response[start_index + len(start_of_response_tag):].strip()

    # O Mistral muitas vezes termina a resposta do modelo com </s>.
    # Remova esse token se ele estiver presente no final.
    if model_raw_answer.endswith("</s>"):
        model_answer = model_raw_answer[:-len("</s>")].strip()
    else:
        model_answer = model_raw_answer.strip()

    if "Answer:" in model_answer:
        model_answer = model_answer.split("Answer: ")[1].strip()

  except:
    model_answer = "Could not parse response."

  return model_answer

Gerando as respostas de SQL e guardando num arquivo jsonl

In [ ]:
import json

output_filename = "respostas_fewshot.jsonl"
with open(output_filename, 'w', encoding='utf-8') as f:
    for i in range(0, 200):

        print("Iteração "  + str(i))

        # Cria o objeto
        db_id = spider_dataset['validation'][i]['db_id']
        query = spider_dataset['validation'][i]['query']
        question = spider_dataset['validation'][i]['question']

        # Gera a query usando sua função
        generated_query = generate_response(question)

        # Cria o objeto no formato desejado
        output_object = {
            'db_id': db_id,
            'response_query': generated_query,
            'original_query': query,
            'question': question
        }

        # Escreve o objeto JSON na linha, seguido por uma quebra de linha
        f.write(json.dumps(output_object, ensure_ascii=False) + '\n')

print("Processo concluído!")

Iteração 71
Iteração 72
Iteração 73
Iteração 74
Iteração 75
Iteração 76
Iteração 77
Iteração 78
Iteração 79
Iteração 80
Iteração 81
Iteração 82
Iteração 83
Iteração 84
Iteração 85
Iteração 86
Iteração 87
Iteração 88
Iteração 89
Iteração 90
Iteração 91
Iteração 92
Iteração 93
Iteração 94
Iteração 95
Iteração 96
Iteração 97
Iteração 98
Iteração 99
Iteração 100
Iteração 101
Iteração 102
Iteração 103
Iteração 104
Iteração 105
Iteração 106
Iteração 107
Iteração 108
Iteração 109
Iteração 110
Iteração 111
Iteração 112
Iteração 113
Iteração 114
Iteração 115
Iteração 116
Iteração 117
Iteração 118
Iteração 119
Iteração 120
Iteração 121
Iteração 122
Iteração 123
Iteração 124
Iteração 125
Iteração 126
Iteração 127
Iteração 128
Iteração 129
Iteração 130
Iteração 131
Iteração 132
Iteração 133
Iteração 134
Iteração 135
Iteração 136
Iteração 137
Iteração 138
Iteração 139
Iteração 140
Iteração 141
Iteração 142
Iteração 143
Iteração 144
Iteração 145
Iteração 146
Iteração 147
Iteração 148
Iteração 149
It